# 5. Catalyst + Oscar

**TODO**
- Write motivation
- Getting CRN equations at steady state
- (can solve for steady state equations also?)
- CRN -> GB simple example
- Cholesterol example
- Independent decomposition

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Symbolics")

  Activating project at `~/Documents/repos/JuliaTutorials`
   Resolving package versions...


In [ ]:
using Symbolics
using Catalyst
using Oscar

## Catalyst to Oscar

### Aside: Symbolics.jl
Catalyst uses `Symbolics.jl` as its computer algebra system. This is slightly different to Oscar.

In [ ]:
@variables x y z

In [ ]:
x isa Number  # true

f(t) = t^2 + t
sym_vec = f.([2x, y^2 + 1, x + z + y])

In [ ]:
using Symbolics: expand
expand.(sym_vec)

### Getting the equations